<a href="https://colab.research.google.com/github/sofiazhuravleva/CL_HSE/blob/main/%D0%94%D0%BE%D0%BC%D0%B0%D1%88%D0%BD%D0%B5%D0%B5_%D0%B7%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 4.

## Часть 1. Парсинг телеграм-каналов

У вас есть 2 файла на выбор в [этой папке](https://disk.yandex.ru/d/druM43Rd5bdiQg):
- Канал Системный Блокъ
- Канал Нерусский мир

В них сохранена история каналов за 2024 год в html-формате

Подготовьте таблицу, в которой будет:
- отправитель (у нас это будет канал - но в личных чатах туда попадает никнейм отправителя, как в ДЗ_3)
- текст сообщения (его можно немного посчистить (***по желанию***) - например, удалить из него тэги)
- дата
- (***по желанию***) реакции

**Важно!** В Нерусском мире есть небольшие ошибки - посты без автора (можно отловить их и подумать, что с ними делать, или использовать try / except, чтобы их просто проигнорировать)

Примерный результат выполнения всех заданий вы можете найти в той же [папке](https://disk.yandex.ru/d/druM43Rd5bdiQg)

In [117]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import codecs
import re

In [125]:
f = codecs.open("Системный Блокъ.html", 'r', 'utf-8') # читаем код html из файла
telegram = f.read() # записываем его в переменную
f.close()

soup = BeautifulSoup(telegram)
soup
print(response.text[:3000])

<!DOCTYPE html>
<html lang='ru' prefix="og: http://ogp.me/ns#  ">
<head>
<title>E-mail науки: детские вопросы, на которые вы не смогли ответить</title>
<noscript><meta name="viewport" content="width=device-width, initial-scale=1.0"></noscript>
<script type='text/javascript'>if (!document.cookie.match(/\bdesktop_mode=1/)) document.write('<meta name="viewport" content="width=device-width, initial-scale=1.0">');</script>


<META name="description" content="Научные ответы на вопросы детей. Теперь вы сможете объяснить ребенку то, чего и сами до сих пор не понимали."/>



<link rel="canonical" href="https://elementy.ru/email?page=1" />


<base href="//elementy.ru/">



<meta property="og:title" content="E-mail науки: детские вопросы, на которые вы не смогли ответить"/>
<meta property="og:type" content="website"/>
<meta property="og:description" content="Научные ответы на вопросы детей. Теперь вы сможете объяснить ребенку то, чего и сами до сих пор не понимали."/>
<meta property="og:url" cont

Советы: можно идти по тегам в супе, а можно использовать регулярные выражения к содержимому тега, оставлю одну из возможных подсказок (разбор тегов супом **предпочтительнее**, но задачу можно также решить любым другим способом)

In [122]:
senders = []
texts = []
reactions_list = []
date_list = []

for msg in soup.find_all('div', class_='message'):
    # Пропускаем служебные сообщения
    if 'service' in msg.get('class', []):
        continue
    # 1. Отправитель
    sender_tag = msg.find('div', class_='from_name')
    if not sender_tag:
            continue  # пропускаем, если нет отправителя

    sender = sender_tag.get_text(strip=True)

# Пропускаем, если это не "Системный Блокъ"
    if sender != "Системный Блокъ":
        continue

    
    # 2. Текст сообщения
    text_tag = msg.find('div', class_='text')
    text = text_tag.get_text(separator=' ', strip=True) if text_tag else ""
    cleaned_text = re.sub(r"(🦾|🤖)*(Системный Блокъ|@sysblock)\s*$", "", text)
    
    # 3. Дата
    date_tag = msg.find('div', class_='pull_right date details')
    date = date_tag['title'] if date_tag and date_tag.has_attr('title') else "Неизвестно"
    
    # 4. Реакции
    reactions_block = msg.find('div', class_='reactions')
    reactions = []
    
    if reactions_block:
        for r in reactions_block.find_all('div', class_='reaction'):
            emoji = r.find('div', class_='emoji')
            count = r.find('div', class_='count')
            if emoji and count:
                reactions.append(f"{emoji.text.strip()} {count.text.strip()}")

    reactions_str = ", ".join(reactions)
    
    # Добавляем данные в списки
    senders.append(sender)
    texts.append(text)
    date_list.append(date)
    reactions_list.append(reactions_str)

# Создаём DataFrame
df = pd.DataFrame({
    "Отправитель": senders,
    "Текст": texts,
    "Дата": date_list,
    "Реакции": reactions_list
})
# Сохраняем в CSV файл
df.to_csv('sysblok_output.csv', index=False, encoding='utf-8-sig')
print("✅ Готово! Сохранено в 'sysblok_output.csv'")

# Печатаем результат
print(df.head())


✅ Готово! Сохранено в 'sysblok_output.csv'
       Отправитель                                                                                                Текст                  Дата                          Реакции
0  Системный Блокъ  Что происходит в интернете: Оксана Мороз о цифровом бессмертии и языке онлайн-скорби Дождались Д...  03.01.2024 19:58:07          ❤ 31, 🎄 5, 👍 3, 🔥 2, 👏 1
1  Системный Блокъ  Танцы, эрос и зачатие: о чем писали «Платоновские исследования» за последние 10 лет В 2024 году ...  06.01.2024 21:02:31         👍 14, 🔥 11, ❤ 7, 🍌 1, 👾 1
2  Системный Блокъ  Читаем секретные письма опальной королевы: историческая криптография на примере Марии Стюарт Ист...  10.01.2024 22:27:56         🔥 15, ❤ 12, 👍 8, 👾 3, 👀 1
3  Системный Блокъ  От Эдисона до Spotify: история форматов музыки Какой была первая песня, записанная на носитель? ...  11.01.2024 19:57:30              🔥 20, 👍 16, ❤ 9, 👻 1
4  Системный Блокъ  Больше, чем энциклопедия: Википедии 23 года! Почему Википедия 

## Часть 2

Парсинг сайта, на выбор:
- [Детские вопросы](https://elementy.ru/email) в журнале "Элементы" (немного легче)
- [Рубрика "Ускользающий мир"](https://www.nkj.ru/special/mir/) журнала "Наука и жизнь"

Общий алгоритм:
- смотрим на 1 и последнюю страницу и с помощью range(от, до+1) генерируем ссылки на страницы
- проходим по каждой странице и парсим ссылки на новости (важно собрать нужные и очистить от лишних!). На момент начала задания было 88 Детских вопроса и 213 текстов Ускользающего мира (ближе к дедлайну их может стать немного больше)
- переходим к каждой новости и парсим из нее:
  - заголовок, автор ответа, текст, ссылку для Детских вопросов
  - заголовок, дату, текст и ссылку для Ускользающего мира (***опционально*** - также автора, источник, тэги, но учтите, что они есть не в каждой статье)
- сохраняем результаты в датафрейм, не забудьте озаглавить столбцы

Поскольку мы парсим реальные сайты, вежливый парсинг с time и задержкой в несколько секунд будет плюсом!

In [3]:
pip install requests



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install tqdm



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [53]:
url = 'https://elementy.ru/email?page=5'
response = requests.get(url)
print(response.status_code) 
print(response.text[:1000])


200
<!DOCTYPE html>
<html lang='ru' prefix="og: http://ogp.me/ns#  ">
<head>
<title>E-mail науки: детские вопросы, на которые вы не смогли ответить</title>
<noscript><meta name="viewport" content="width=device-width, initial-scale=1.0"></noscript>
<script type='text/javascript'>if (!document.cookie.match(/\bdesktop_mode=1/)) document.write('<meta name="viewport" content="width=device-width, initial-scale=1.0">');</script>


<META name="description" content="Научные ответы на вопросы детей. Теперь вы сможете объяснить ребенку то, чего и сами до сих пор не понимали."/>



<link rel="canonical" href="https://elementy.ru/email?page=5" />


<base href="//elementy.ru/">



<meta property="og:title" content="E-mail науки: детские вопросы, на которые вы не смогли ответить"/>
<meta property="og:type" content="website"/>
<meta property="og:description" content="Научные ответы на вопросы детей. Теперь вы сможете объяснить ребенку то, чего и сами до сих пор не понимали."/>
<meta property="og:url" 

In [54]:
with open("page5.html", "w", encoding="utf-8") as f:
    f.write(response.text)


In [111]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36',
    'Referer': 'https://elementy.ru/',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7'
}
# Сбор всех ссылок на вопросы
def collect_all_question_links():
    all_links = set()
    page = 0
    while True:
        url = 'https://elementy.ru/email' if page == 0 else f'https://elementy.ru/email?page={page}'
        print(f'🔗 Получаем ссылки с {url}')
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = {
            'https://elementy.ru' + a.get('href')
            for a in soup.find_all('a', class_='nohover')
            if a.get('href', '').startswith('/email/')
        }
        print(f'  Найдено ссылок: {len(links)}')
        if not links:
            break
        all_links.update(links)
        page += 1
        time.sleep(1.5)
    print(f'\n📌 Всего собрано ссылок: {len(all_links)}')
    return list(all_links)

# Парсинг отдельной страницы
def parse_question(url):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Заголовок
        title_tag = soup.find("h1", itemprop="name")
        title = title_tag.get_text(strip=True) if title_tag else "—"

        # Автор — три варианта
        author = "—"

        # Вариант 1: обычный автор из sublink
        sublink_div = soup.find("div", class_="sublink")
        if sublink_div:
            raw = sublink_div.get_text(strip=True)
            if '•' in raw:
                author_candidate = raw.split('•')[0].strip()
                if len(author_candidate) > 1:
                    author = author_candidate

        # Вариант 2: из <p class="small">Ответил <i>Автор</i>
        if author == "—":
            small_block = soup.find("p", class_="small")
            if small_block:
                match = re.search(r'Ответил\s+<i>(.+?)</i>', str(small_block))
                if match:
                    author = match.group(1).strip()

        # Вариант 3: автор(ы) в <a> и <i> внутри текста
        if author == "—":
            candidates = soup.select('div[itemprop="text"] a, div[itemprop="text"] i')
            authors_found = set()
            for tag in candidates:
                text_tag = tag.get_text(strip=True)
                if text_tag and 3 <= len(text_tag) <= 50:
                    authors_found.add(text_tag)
            if authors_found:
                author = ", ".join(sorted(authors_found))

        # Текст статьи с расширенной фильтрацией
        text_block = soup.find("div", itemprop="text")
        text = ""
        if text_block:
            paragraphs = []
            for p in text_block.find_all("p"):
                text_p = p.get_text(strip=True)
                only_tags = [child for child in p.children if hasattr(child, 'name')]

                # Расширенный фильтр: убираем подписи и источники
                if (
                    (author != "—" and author in text_p)
                    or text_p.startswith("—")
                    or (len(only_tags) == 1 and only_tags[0].name in {"i", "a", "b"})
                    or (len(text_p) <= 100 and '.' not in text_p)
                    or re.search(r'(источник|по материалам|журнал|сайт|www\.)', text_p.lower())
                ):
                    continue

                paragraphs.append(text_p)

            text = "\n".join(paragraphs)

        return {
            "Заголовок": title,
            "Автор": author,
            "Текст": text,
            "Ссылка": url
        }

    except Exception as e:
        print(f"❌ Ошибка {url}: {e}")
        return None

# Главная функция
def main():
    all_links = collect_all_question_links()
    data = []

    for i, url in enumerate(all_links, 1):
        print(f"📄 Парсим новость {i}/{len(all_links)}")
        result = parse_question(url)
        if result:
            data.append(result)
        time.sleep(1.5)

    with open("детские_вопросы.csv", "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["Заголовок", "Автор", "Текст", "Ссылка"])
        writer.writeheader()
        writer.writerows(data)

    print("\n✅ Готово! Сохранено в файл: детские_вопросы.csv")

# Запуск
if __name__ == "__main__":
    main()


🔗 Получаем ссылки с https://elementy.ru/email
  Найдено ссылок: 20
🔗 Получаем ссылки с https://elementy.ru/email?page=1
  Найдено ссылок: 20
🔗 Получаем ссылки с https://elementy.ru/email?page=2
  Найдено ссылок: 20
🔗 Получаем ссылки с https://elementy.ru/email?page=3
  Найдено ссылок: 20
🔗 Получаем ссылки с https://elementy.ru/email?page=4
  Найдено ссылок: 8
🔗 Получаем ссылки с https://elementy.ru/email?page=5
  Найдено ссылок: 0

📌 Всего собрано ссылок: 88
📄 Парсим новость 1/88
📄 Парсим новость 2/88
📄 Парсим новость 3/88
📄 Парсим новость 4/88
📄 Парсим новость 5/88
📄 Парсим новость 6/88
📄 Парсим новость 7/88
📄 Парсим новость 8/88
📄 Парсим новость 9/88
📄 Парсим новость 10/88
📄 Парсим новость 11/88
📄 Парсим новость 12/88
📄 Парсим новость 13/88
📄 Парсим новость 14/88
📄 Парсим новость 15/88
📄 Парсим новость 16/88
📄 Парсим новость 17/88
📄 Парсим новость 18/88
📄 Парсим новость 19/88
📄 Парсим новость 20/88
📄 Парсим новость 21/88
📄 Парсим новость 22/88
📄 Парсим новость 23/88
📄 Парсим новость 